# Tutorial de Big Data
## Bienvenidos a la clase 8 

**Objetivo:** 
Que se familiaricen con la técnica de K-fold Cross Validation

### Temario:
- Preguntas de GitHub: Recuerden que todos los miembros del equipo tienen que hacer al menos un push para el TP3
- Practica para no repetir código
- K-fold cross validation


In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error

In [ ]:
auto = pd.read_csv("Auto.csv")

# Ver datos sobre la base en [link](https://rdrr.io/cran/ISLR/man/Auto.html)

# Guardo los vectores de variable dependiente y de variable independiente respectivamente:
y = auto['mpg']
X = auto['horsepower']
X = np.array(X).reshape((-1, 1))

# Parto la base en dos y transformo el vector x: 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)


In [ ]:
# Recuerdan que hemos visto Regresiones Polinomicas, las cuales implican una 
# transformación polinomica de las X, para luego implementar la regresión?

# Veamos un modelo cuadrático:
poly = PolynomialFeatures(degree = 2, include_bias=False) 

# Recordemos en esta instancia setear include_bias a False dado que en la 
# regresión lineal se incluirá la columna de 1s.

#print(x_train)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)  
  
#print(x_train_poly)

model = LinearRegression().fit(x_train_poly, y_train) 

print('Intercepto:', model.intercept_)
print('Pendiente:', model.coef_)

# Calculamos el Error Cuadrático Medio
y_predpoly = model.predict(x_test_poly)
ecm2 = mean_squared_error(y_predpoly, y_test)
ecm2

In [ ]:
# Veamos un modelo cubico:
poly = PolynomialFeatures(degree = 3, include_bias=False) 

x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)  
  
model = LinearRegression().fit(x_train_poly, y_train) 
y_predpoly = model.predict(x_test_poly)

ecm3 = mean_squared_error(y_predpoly, y_test)
ecm3

In [ ]:
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
X_seq_poly = poly.fit_transform(X_seq)  

plt.figure()
plt.scatter(x_train, y_train)
plt.plot(X_seq, model.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree 3")
plt.show()

In [ ]:
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
X_seq_poly = poly.fit_transform(X_seq)  

plt.figure()
plt.scatter(x_test, y_test)
plt.plot(X_seq, model.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree 3")
plt.show()

In [ ]:
# Ahora supongamos que cambio la muestra y hago todo lo mismo.
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 500)

# Que error esperarian que obtengamos esta vez?


In [ ]:
# Como podemos repetir el código sin escribirlo por tercera vez?
# Podemos hacer que nuestro código funcione para otros grados?




In [ ]:
ecm2b = transformacion_y_regresion(2, x_train, x_test, y_train, y_test)
ecm3b = transformacion_y_regresion(3, x_train, x_test, y_train, y_test)
ecm4 = transformacion_y_regresion(4, x_train, x_test, y_train, y_test)
ecm5 = transformacion_y_regresion(5, x_train, x_test, y_train, y_test)

In [ ]:
print('Grado2:', ecm2)
print('Grado3:', ecm3)

print('\nGrado2:', ecm2b)
print('Grado3:', ecm3b)
print('Grado4:', ecm4)
print('Grado5:', ecm5)

In [ ]:
# Otra forma de hacer lo mismo pero aprovechando el metodo pipeline:
from sklearn.pipeline import make_pipeline

grado = 3
polyreg = make_pipeline(PolynomialFeatures(grado),LinearRegression())
modelo = polyreg.fit(x_train, y_train)
y_pred_poly = modelo.predict(x_test)

    
# Calculamos el Error Cuadrático Medio
ecm = mean_squared_error(y_pred_poly, y_test)
print(ecm)

###  K-FOLD CROSS-VALIDATION  


In [ ]:
y = auto['mpg']
X = auto['horsepower']
X = np.array(X).reshape((-1, 1))

from sklearn.model_selection import KFold

K = 5

ecms = pd.DataFrame(columns=["grado", "particion", "ecm"])

for grado in range(2, 10):   

    kf = KFold(n_splits=K, shuffle=True, random_state=100)
    
    # El método kf.split aplicado a X nos da los conjuntos de índices que necesitamos para
    # partir nuestros conjunto de datos en training y testing en cada iteración.
    #  OXXXX
    #  XOXXX
    #  XXOXX
    #  XXXOX
    #  XXXXO
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):   
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        ecm, modelo = transformacion_y_regresion(grado, x_train, x_test, y_train, y_test)
            
        ecms = ecms.append({"grado": grado, "particion": i, "ecm": ecm}, ignore_index=True)            
    
ecms = ecms.astype({"grado":int, "particion":int})
ecms

In [ ]:
# Otra opción podría ser visualizandolo en un boxplot
import seaborn as sns
sns.set()
ss = sns.boxplot(data=mses, x="grado", y="mse")


In [ ]:
#Una opción para ver el mejor modelo sería sacar el error promedio para cada grado:
ecms.groupby('grado').agg({'ecm':'mean'})

In [ ]:
#Construyan una función que les permita seleccionar 


In [ ]:
#Fianalmente construimos el modelo polinomial de grado 6 y lo graficamos 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

ecm, modelo = transformacion_y_regresion(grado, x_train, x_test, y_train, y_test)
        
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
poly = PolynomialFeatures(degree = grado, include_bias=False) 
X_seq_poly = poly.fit_transform(X_seq)  

plt.figure()
plt.scatter(x_train, y_train)
plt.plot(X_seq, modelo.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree {}".format(grado))
plt.show()
